In [7]:
import numpy as np
import glob
import errno
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.optim as optim
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

In [15]:
path = os.getcwd()+"/data/text_data/train/pos/*.txt"
files = glob.glob(path)
counter = 0
corpus = ""

for name in files:
    try:
        with open(name, 'r') as f:
            if counter < 50:
                #print(review)
                counter += 1
                review = f.read().replace('\n', '').replace('<br /><br />', ' ').replace('.', ' .').replace('!', ' !').replace('?', ' ?')
                #review = "i ate a dog with some ketchup . it was very tasty . then i ate a cat with some mustard . it was also very tasty ."
                review = review.lower()
                #print(review)
                corpus += review
                corpus += " "
                #review = review.split()
                #review = ['<start>'] + review + ['<end>']
    except IOError as exc:
        if exc.errno != errno.EISDIR:
            raise
            
    
print(len(corpus))
#print(corpus)

55938


In [16]:
chars = sorted(list(set(corpus))) # getting all unique chars
num_unique_chars = len(chars)
print('total chars: ', num_unique_chars)
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars:  62


In [20]:
maxlen = 40
step = 40
sentences = []
sentences_shifted = []
next_chars = []
for i in range(0, len(corpus) - maxlen, step):
    sentences.append(corpus[i: i + maxlen])
    sentences_shifted.append(corpus[i+1: i+1+maxlen])
    next_chars.append(corpus[i + maxlen])
    
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.float)
y = np.zeros((len(sentences), maxlen), dtype=np.float)

print(sentences[0])
print(sentences_shifted[0])

for i in range(0, len(sentences)):
    sentence = sentences[i]
    sentence_shifted = sentences_shifted[i]
    for t in range(0, len(sentence)):
        char = sentence[t]
        char_shifted = sentence_shifted[t]
        x[i, t, char_indices[char]] = 1
        y[i, t] = char_indices[char_shifted]

print(x.shape)
print(y.shape)
'''
for i in range(0, len(sentences)):
    sentence = sentences[i]
    for t in range()

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i] = char_indices[next_chars[i]]
'''

for a movie that gets no respect there s
or a movie that gets no respect there su
(1398, 40, 62)
(1398, 40)


'\nfor i in range(0, len(sentences)):\n    sentence = sentences[i]\n    for t in range()\n\nfor i, sentence in enumerate(sentences):\n    for t, char in enumerate(sentence):\n        x[i, t, char_indices[char]] = 1\n    y[i] = char_indices[next_chars[i]]\n'

In [91]:
print(x.shape)
print(y.shape)

(18613, 100, 62)
(18613,)


In [92]:
print(x[0])
print(y[0])

[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]
40.0


In [93]:
torch.from_numpy(y).long()

tensor([40,  0, 44,  ..., 40, 40,  1])

In [94]:
class LSTM(nn.Module):
    def __init__(self, hidden_size, layer_dim=1):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.layer_dim = layer_dim
        self.lstm1 = nn.LSTM(input_size=num_unique_chars, hidden_size=hidden_size, num_layers=layer_dim, batch_first=True, bidirectional=True, dropout=0.4)
        self.fc1 = nn.Linear(hidden_size*2, 50) # hidden_size *2 because of bidirectional
        self.fc2 = nn.Linear(50, 50)
        self.fc3 = nn.Linear(50, num_unique_chars)
        self.dropout = nn.Dropout(p=0.4)
        
    def forward(self, inputs):
        out, (hn, cn) = self.lstm1(inputs)
        hn_flat = torch.cat((hn[0], hn[1]), dim=1)
        out = F.relu(self.fc1(hn_flat))
        out = self.dropout(out)
        out = F.relu(self.fc2(out))
        out = self.dropout(out)
        out = self.fc3(out)
        return out
    
net = LSTM(hidden_size=20, layer_dim=3)

In [95]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.01)

In [ ]:
for epoch in range(40):
    optimizer.zero_grad()
    
    inputs = torch.from_numpy(x).float()
    target = torch.from_numpy(y).long()
    
    outputs = net(inputs)
    #print(outputs.shape)
    #print(target.shape)
    #print(outputs)
    #print(target)
    loss = criterion(outputs, target)
    
    print("loss: ", loss)
    
    loss.backward()
    optimizer.step()

print("Finished Training")

loss:  tensor(4.1467, grad_fn=<NllLossBackward>)
loss:  tensor(4.0651, grad_fn=<NllLossBackward>)
loss:  tensor(3.8700, grad_fn=<NllLossBackward>)
loss:  tensor(3.3345, grad_fn=<NllLossBackward>)
loss:  tensor(4.3444, grad_fn=<NllLossBackward>)
loss:  tensor(3.3886, grad_fn=<NllLossBackward>)
loss:  tensor(3.1984, grad_fn=<NllLossBackward>)
loss:  tensor(3.2891, grad_fn=<NllLossBackward>)
loss:  tensor(3.3694, grad_fn=<NllLossBackward>)
loss:  tensor(3.3942, grad_fn=<NllLossBackward>)


In [77]:
def generate_next_char(sentence):
    #sentence is 40 characters
    sentence_preprocess = sentence.replace('\n', '').replace('<br /><br />', ' ').replace('.', ' .').replace('!', ' !').replace('?', ' ?')
    sentence_preprocess = sentence_preprocess.lower()
    sentence_trunc = [sentence_preprocess[-40:]]
    x = np.zeros((len(sentence_trunc), len(sentence_trunc[0]), len(chars)), dtype=np.bool)
    #y = np.zeros((len(sentences)), dtype=np.float)
    for i, sentence in enumerate(sentence_trunc):
        for t, char in enumerate(sentence):
            x[i, t, char_indices[char]] = 1
        #y[i] = char_indices[next_chars[i]]
    
    inputs = torch.from_numpy(x).float()
    outputs = net(inputs)
    max_indice = torch.argmax(outputs.view(outputs.size(1))).item()
    #print(sentence_preprocess)
    new_sentence = sentence_preprocess + indices_char[max_indice]
    return new_sentence

In [78]:
def multi_generate(sentence, n):
    for i in range(n):
        sentence = generate_next_char(sentence)
    return sentence

In [79]:
sentence = "If you like adult comedy cartoons, like South Park, then this is nearly a similar format about the small adventures of three teenage girls at Bromwell High."
print(sentence[-40:])

of three teenage girls at Bromwell High.


In [80]:
generate_next_char(sentence)

'if you like adult comedy cartoons, like south park, then this is nearly a similar format about the small adventures of three teenage girls at bromwell high . '

In [81]:
multi_generate(sentence, 100)

'if you like adult comedy cartoons, like south park, then this is nearly a similar format about the small adventures of three teenage girls at bromwell high                                                                                                    .                                                                                                    '